In [153]:
import pandas as pd



pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

referendum_data = pd.read_csv(
    'data/EU-referendum-result-data.csv'
)

ages = pd.read_csv(
    'data/ages.csv'
)
density = pd.read_csv(
    'data/density.csv'
)
# ethnic_groups = pd.read_csv(
#     'data/ethnic_groups.csv'
# )
health = pd.read_csv(
    'data/health.csv'
)
sex = pd.read_csv(
    'data/sex.csv'
)

areas_lat_lon = pd.read_csv(
    'data/areas_lat_lon.csv'
)


# health['Very good health'] = health['Very good health'].apply(lambda x: x.replace(",", ""))
# health['Good health'] = health['Good health'].apply(lambda x: x.replace(",", ""))
# health['Fair health'] = health['Fair health'].apply(lambda x: x.replace(",", ""))
# health['Very bad health'] = health['Very bad health'].apply(lambda x: x.replace(",", ""))
# health['Bad health'] = health['Bad health'].apply(lambda x: x.replace(",", ""))
# health['Area code']

# sex['Males'] = sex['Males'].apply(lambda x: x.replace(",", ""))
# sex['Females'] = sex['Females'].apply(lambda x: x.replace(",", ""))
# sex['Area code']


# density['Density']
# density['Area code']


health = health[
    ['Very good health',
    'Good health',
    'Fair health',
    'Very bad health',
    'Bad health',
    'Area code',]
]
sex = sex[['Males', 'Females', 'Area code']]
ages = ages[['Mean age', 'Area code']]
density = density[['Area code', 'Density']]
referendum_data = referendum_data[[
    'Area_Code', 'Area', 'Remain', 'Leave'
]]
referendum_data = referendum_data.rename(columns={'Area_Code': 'Area code'})
areas_lat_lon = areas_lat_lon[[
    'lad17cd', 'lat', 'long'
]]
areas_lat_lon=areas_lat_lon.rename(columns={'lad17cd': 'Area code'})


ward_to_district = pd.read_csv('data/ward_to_local_district.csv')
ward_to_district=ward_to_district.rename(columns={'WD18CD': 'ward_id', 'LAD18CD': 'Area code'})
ward_to_district=ward_to_district[['ward_id', 'Area code']]
# LAD18CD
# WD18CD

ward_codes = pd.read_csv('data/ward-codes-w-30-45.csv')
ward_codes=ward_codes.rename(columns={'ward': 'ward_id'})
ward_codes = ward_codes.merge(
    ward_to_district, on='ward_id', how='left'
)
# ward_codes=ward_codes[ward_codes['Area code'].notnull()]
# ward_codes.groupby("Area code").mean().sort_values("30-45w", ascending=False)
ward_codes['number'] = ward_codes['30-45w'] * ward_codes['pop'] 

ward_codes = ward_codes.drop_duplicates()
ward_result = ward_codes.groupby("Area code").sum().sort_values("number", ascending=False).reset_index()
ward_result = ward_result[['Area code', 'number']]
ward_result.head()
ward_codes.head()

ward_to_district_2 = pd.read_csv('data/ward_to_local_district.csv')
ward_to_district_2
ward_codes = pd.read_csv('data/ward-codes-w-30-45.csv')
ward_codes.sum()

# ward_codes = pd.read_csv('data/ward-codes-w-30-45.csv')
# ward_codes=ward_codes.rename(columns={'ward': 'ward_id'})
# ward_codes = ward_codes.merge(
#     ward_to_district, on='ward_id', how='left'
# )
referendum_data = referendum_data.merge(
    density, on='Area code', how='left'
)
referendum_data = referendum_data.merge(
    sex, on='Area code', how='left'
)

referendum_data = referendum_data.merge(
    health, on='Area code', how='left'
)

referendum_data = referendum_data.merge(
    ages, on='Area code', how='left'
)


referendum_data = referendum_data.merge(
    areas_lat_lon, on='Area code', how='left'
)

def _clean(x):
    try:
        return int(x.replace(",", ""))
    except:
        return x

# referendum_data.to_csv("the_real_data.csv")
# referendum_data['Area code']
referendum_data[referendum_data['Area code'] == 'E07000220']
res = referendum_data.merge(ward_result, on="Area code", how="left")

res['remain_pct'] = res['Remain'] / (res['Remain'] + res['Leave'])
res['women_voters_pct'] = res['Females'].apply(_clean) / (res['Females'].apply(_clean) + res['Males'].apply(_clean))

# .apply(_clean)

res['Very good health'] = res['Very good health'].apply(_clean)
res['Good health'] = res['Good health'].apply(_clean)
res['Fair health'] = res['Fair health'].apply(_clean)
res['Bad health'] = res['Bad health'].apply(_clean)
res['Very bad health'] = res['Very bad health'].apply(_clean)

res['very_good_health_pct'] = res['Very good health'] / (res['Very good health'] + res['Good health'] +res['Bad health'] + res['Fair health'] + res['Very bad health'])
res['very_bad_health_pct'] =  res['Very bad health'] / (res['Very good health'] + res['Good health'] + res['Bad health']+res['Fair health'] + res['Very bad health'])

res = res.sort_values("number", ascending=False)
res





,Area code,Area,Remain,Leave,Density,Males,Females,Very good health,Good health,Fair health,Very bad health,Bad health,Mean age,lat,long,number,remain_pct,women_voters_pct,very_good_health_pct,very_bad_health_pct
280,E06000052,Cornwall,140540,182665,1.5,"257,805","274,468",240759.0,178805.0,79181.0,7621.0,25907.0,43.3,50.450230,-4.642490,49530.0,0.434832,0.515653,0.452322,0.014318
282,E06000054,Wiltshire,137258,151637,1.4,"231,754","239,227",234318.0,160362.0,57221.0,4106.0,14974.0,40.8,51.328831,-1.926610,47863.0,0.475114,0.507933,0.497511,0.008718
125,E06000047,County Durham,113521,153877,2.3,"251,280","261,962",217373.0,171564.0,82404.0,9333.0,32568.0,41.0,54.685131,-1.840500,40738.0,0.424539,0.510406,0.423529,0.018184
322,W06000015,Cardiff,101788,67816,24.7,"169,893","176,197",174329.0,107477.0,42014.0,5525.0,16745.0,36.5,51.502541,-3.222120,36874.0,0.600151,0.509107,0.503710,0.015964
136,E06000049,Cheshire East,107962,113163,3.2,"181,409","188,718",181595.0,123065.0,47306.0,3914.0,14247.0,42.0,53.167931,-2.292990,36016.0,0.488240,0.509874,0.490629,0.010575
211,E06000042,Milton Keynes,63393,67063,8.1,"123,116","125,705",124826.0,86778.0,26929.0,2333.0,7955.0,36.1,52.072411,-0.740700,33680.0,0.485934,0.505203,0.501670,0.009376
5,E06000056,Central Bedfordshire,69670,89134,3.6,"126,044","128,337",125019.0,90360.0,29202.0,2128.0,7672.0,39.6,51.999031,-0.477540,31257.0,0.438717,0.504507,0.491464,0.008365
362,E06000011,East Riding of Yorkshire,78779,120136,1.4,"163,388","170,791",151003.0,116859.0,48307.0,4133.0,13877.0,43.5,53.881222,-0.662030,29451.0,0.396044,0.511076,0.451863,0.012368
126,E06000057,Northumberland,82022,96699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.300369,-2.075210,28600.0,0.458939,NaN,NaN,NaN
377,E08000032,Bradford,104575,123913,14.3,"257,132","265,320",234133.0,186031.0,71580.0,6940.0,23768.0,36.2,53.843819,-1.873890,28293.0,0.457683,0.507836,0.448143,0.013284


In [156]:
import numpy as np
res = res.sort_values("remain_pct", ascending=False)
_res = []
for _, row in res.iterrows(): 
    if np.isnan(row['Mean age']) or np.isnan(row['remain_pct']):
        pass
    else:
        x = [row['remain_pct'] * 100, row['Mean age']]
        print(x)
        _res.append(x)

# xx=pd.DataFrame(_res)
# xx.head()
_res[3]

[78.62236126377498, 33.7]
[78.48041706660645, 32.4]
[75.57300228634053, 34.2]
[75.28983859968174, 41.4]
[75.21653543307086, 34.5]
[75.03166882014644, 34.5]
[74.94244899246318, 35.7]
[74.43538094445368, 38.5]
[74.31858106193687, 41.0]
[73.84556826242668, 35.6]
[72.81200290343006, 33.7]
[71.39833905152788, 42.2]
[70.26742681661146, 34.3]
[70.0231798808604, 34.7]
[69.8585235352245, 34.6]
[69.29397275885522, 38.3]
[68.96516445854007, 36.7]
[68.69142658799028, 37.8]
[68.61973751491392, 37.4]
[67.71508619808176, 40.2]
[67.45599852173511, 30.9]
[64.81294466066788, 40.6]
[64.6044042751596, 41.0]
[63.796578634554756, 41.5]
[63.161131611316115, 42.6]
[63.083054516827396, 40.7]
[62.940059940059946, 36.3]
[62.229480913849955, 36.8]
[62.05903052696402, 40.4]
[61.9944470152707, 40.3]
[61.73345881940948, 36.5]
[61.66480583934069, 39.0]
[61.60783393925179, 37.0]
[61.10453234524515, 38.7]
[61.092855824256965, 42.6]
[60.56760308854937, 43.9]
[60.40217121463222, 35.4]
[60.36399853330294, 33.2]
[60.230284

[75.28983859968174, 41.4]

In [151]:
df = pd.read_csv("data/ages2.csv")


/home/harry/demographics/pyenv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (21,22,23,24,25,26,27,28,29,30,31,32,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Ward Code 1,All Ages,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90+
0,E05000026,"15,460",334,350,330,367,323,334,278,280,271,238,232,218,202,169,175,150,153,160,140,144,132,153,183,200,240,303,303,324,368,414,405,412,435,402,385,379,336,353,291,264,262,238,245,224,191,186,186,164,149,166,142,164,120,113,134,112,87,95,81,83,103,99,66,66,75,53,53,49,43,42,65,41,50,31,37,35,24,21,28,30,15,18,17,13,17,22,13,21,12,10,19
1,E05000027,"10,995",191,202,199,206,198,188,221,174,151,203,182,198,152,141,155,139,142,128,145,158,123,125,151,131,155,148,144,165,159,169,162,160,172,185,150,173,185,154,147,139,167,147,161,162,172,180,170,166,145,152,164,151,154,149,155,131,139,116,122,115,96,79,67,78,73,63,64,50,61,48,62,59,53,28,49,39,30,35,33,38,28,26,25,29,19,22,33,15,14,25,66
2,E05000028,"14,862",292,306,302,281,323,285,275,282,267,303,293,202,242,195,190,205,177,174,162,185,156,167,202,198,215,204,198,221,242,241,264,283,319,313,292,248,257,250,239,226,210,203,209,200,190,221,195,200,192,182,161,171,194,167,141,150,160,124,140,101,119,108,110,93,90,82,93,82,63,69,54,52,60,55,49,32,48,37,49,38,41,30,29,25,21,22,16,19,13,13,63
3,E05000029,"11,265",191,200,191,171,188,204,172,183,193,212,164,176,175,176,165,159,125,142,149,116,113,115,145,152,133,158,139,165,154,193,155,183,188,159,178,171,146,153,139,154,158,133,145,153,156,128,175,152,131,132,145,137,136,145,113,138,115,115,111,105,96,82,81,74,83,87,86,74,80,60,89,71,82,61,68,64,49,70,45,72,49,55,51,37,39,40,30,38,27,26,131
4,E05000030,"11,473",173,184,154,193,162,168,152,152,173,160,147,139,147,130,110,146,153,140,157,156,146,143,201,187,195,168,186,178,184,187,161,127,171,159,134,151,140,155,137,138,125,130,119,157,147,148,139,139,151,159,199,153,162,159,162,145,146,154,126,123,111,115,96,93,107,88,91,77,87,93,101,78,49,53,77,70,60,50,65,60,63,55,48,41,50,40,34,35,28,26,145
5,E05000031,"12,878",218,257,240,215,214,246,228,249,209,227,249,211,207,168,159,162,165,174,152,183,132,161,169,206,185,158,202,209,212,194,212,241,204,201,204,220,210,239,224,186,194,189,197,184,209,185,182,173,155,165,152,147,150,118,123,153,134,133,113,110,95,86,111,74,107,80,74,61,56,58,51,66,49,46,45,44,38,44,49,48,32,35,35,26,25,23,18,33,14,18,69
6,E05000032,"12,806",298,294,288,288,287,309,256,291,248,262,261,264,219,213,185,182,159,157,145,119,109,113,158,159,162,163,203,219,278,262,247,232,289,256,243,221,247,245,198,229,216,202,213,195,207,166,150,176,145,116,129,99,109,115,113,91,85,97,84,79,72,72,78,63,67,52,47,43,41,36,52,43,47,39,35,30,21,24,14,16,12,24,18,17,12,14,10,11,7,7,37
7,E05000033,"12,178",174,216,209,210,222,217,243,236,218,238,219,173,186,145,161,158,170,175,134,130,139,130,175,161,154,171,154,155,167,180,169,197,190,184,168,215,186,186,195,170,208,180,185,173,190,159,169,187,152,177,147,158,160,131,141,127,107,117,131,104,96,93,94,109,85,87,87,80,84,74,90,57,58,51,50,52,31,43,39,42,38,31,29,30,34,21,38,32,25,28,87
8,E05000034,"12,171",209,248,207,211,221,222,215,225,226,226,208,221,182,141,167,188,171,173,168,149,144,126,148,157,161,174,199,181,172,196,175,173,178,174,168,161,164,159,152,149,183,175,196,186,154,159,183,168,169,165,148,149,144,124,134,140,131,109,101,109,100,98,91,98,81,64,89,59,71,75,73,61,66,47,58,51,44,54,42,55,57,47,36,37,26,43,20,27,29,36,120
9,E05000035,"12,784",201,214,192,190,233,203,219,199,188,185,201,170,159,136,169,178,193,155,171,166,160,223,200,226,224,221,205,232,216,256,204,243,225,226,211,204,184,200,172,180,186,163,182,184,143,135,158,162,152,152,150,133,118,146,165,149,131,122,135,130,117,86,100,103,111,80,80,76,63,81,77,56,56,57,42,43,39,58,38,65,38,42,38,30,33,44,30,29,23,21,98
